In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holidays

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

In [3]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [4]:
data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,2.302585


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 455163 entries, 48321 to 928462
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 455163 non-null  category      
 1   counter_name               455163 non-null  category      
 2   site_id                    455163 non-null  int64         
 3   site_name                  455163 non-null  category      
 4   bike_count                 455163 non-null  float64       
 5   date                       455163 non-null  datetime64[us]
 6   counter_installation_date  455163 non-null  datetime64[us]
 7   counter_technical_id       455163 non-null  category      
 8   latitude                   455163 non-null  float64       
 9   longitude                  455163 non-null  float64       
 10  log_bike_count             455163 non-null  float64       
dtypes: category(4), datetime64[us](2), float64(4), int64(

In [6]:
data.nunique(axis=0)

counter_id                     56
counter_name                   56
site_id                        30
site_name                      30
bike_count                    977
date                         8230
counter_installation_date      22
counter_technical_id           30
latitude                       30
longitude                      30
log_bike_count                977
dtype: int64

In [7]:
# Check whether counter_ids are similar to counter_name in terms of information
counter_unique_names = data.groupby('counter_id', observed=False)['counter_name'].nunique()

inconsistent_entries = counter_unique_names[counter_unique_names > 1]

if inconsistent_entries.empty:
    print("All 'counter_id' values have the same corresponding 'counter_name'.")
else:
    print("There are inconsistent entries between 'counter_id' and 'counter_name'.")
    print(inconsistent_entries)

All 'counter_id' values have the same corresponding 'counter_name'.


In [8]:
# Do the same thing with site_ids and site_names
site_unique_names = data.groupby('site_id', observed=False)['site_name'].nunique()

inconsistent_entries = site_unique_names[site_unique_names > 1]

if inconsistent_entries.empty:
    print("All 'site_id' values have the same corresponding 'site_name'.")
else:
    print("There are inconsistent entries between 'site_id' and 'site_name'.")
    print(inconsistent_entries)

All 'site_id' values have the same corresponding 'site_name'.


In [9]:
counter_unique_names = data.groupby('site_id', observed=False)['counter_technical_id'].nunique()

inconsistent_entries = counter_unique_names[counter_unique_names > 1]

if inconsistent_entries.empty:
    print("All 'site_id' values have the same corresponding 'counter_technical_id''.")
else:
    print("There are inconsistent entries between 'site_id' and 'counter_technical_id''.")
    print(inconsistent_entries)

All 'site_id' values have the same corresponding 'counter_technical_id''.


What we learned from this is that there are two columns which are redundant in this dataset. The `counter_id` and the `counter_name` are two and the same thing, and similarly for `site_id` and `site_name`. However, intuitively, the same thing could have been said about the `counter_technical_id` and the rest of the column counter identifications; but what this analysis has shown us is that the `counter_technical_id` is something different, which we should treat as such.

In [10]:
fr_holidays = holidays.FR()

In [11]:
def time_of_day(hour):
    if hour > 3 and hour <= 6:
        return 0
    elif hour > 6 and hour <= 10:
        return 1
    elif hour > 10 and hour <= 13:
        return 2
    elif hour > 13 and hour <= 17:
        return 3
    elif hour > 17 and hour <= 22:
        return 4
    return 5

In [12]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour
    # X.loc[:, "bank_holiday"] = X["date"].apply(lambda x: 1 if x in fr_holidays else 0)
    # X.loc[:, "week-end"] = X["weekday"].apply(lambda x: 1 if x in [5, 6] else 0)
    # X.loc[:, "time_of_day"] = X["hour"].apply(lambda x: time_of_day(x))

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [13]:
date_encoder = FunctionTransformer(_encode_dates, validate=False)
date_encoder.fit_transform(data[["date"]]).head()

,year,month,day,weekday,hour
48321,2020,9,1,1,2
48324,2020,9,1,1,3
48327,2020,9,1,1,4
48330,2020,9,1,1,15
48333,2020,9,1,1,18


In [14]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output=False)

enc.fit_transform(_encode_dates(data[["date"]])[["hour"]])[:5]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0.]])

In [15]:
import problem

X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()
X_final = problem.get_final_test_data()

In [16]:
X_train.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,counter_technical_id,latitude,longitude
400125,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233
408305,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233
87516,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020
98518,100036719-103036719,18 quai de l'Hôtel de Ville SE-NO,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020
875137,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,2020-09-01 01:00:00,2020-07-22,Y2H20073268,48.885290,2.326660


In [17]:
y_train

array([1.60943791, 1.38629436, 0.        , ..., 2.48490665, 1.60943791,
       1.38629436])

For now we do not need to look at the longitude and the latitude, as they are basically one and the same with the `site_id`.

In [18]:
def _encode_categorical(X):
    X = X.copy()

    return X.drop(columns=['site_name', 'counter_technical_id', 'counter_name','latitude', 'longitude'])

In [26]:
def get_estimator():
    date_encoder = FunctionTransformer(_encode_dates)
    date_cols = ["year", "month", "day", "weekday", "hour"]

    categorical_encoder = FunctionTransformer(_encode_categorical)
    categorical_cols = ["counter_id", "site_id", "counter_installation_date"]

    preprocessor = ColumnTransformer(
        [
            ("date", StandardScaler(), date_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ]
    )
    regressor = XGBRegressor(n_estimators=300, max_depth=25, eta=0.1, subsample=0.9, colsample_bytree=0.6)
    # regressor = GradientBoostingRegressor(n_estimators=30, max_depth=25)
    # regressor = Ridge()
    # regressor = LinearRegression()

    pipe = make_pipeline(date_encoder, categorical_encoder, preprocessor, regressor)

    return pipe

In [27]:
pipe = get_estimator()
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_final)

print(f"Test set, RMSE={mean_squared_error(y_test, pipe.predict(X_test), squared=False):.2f}")

Test set, RMSE=0.46


In [28]:
cv = TimeSeriesSplit(n_splits=6)

scores = cross_val_score(
    pipe, X_train, y_train, cv=cv, scoring="neg_root_mean_squared_error"
)
print("RMSE: ", scores)
print(f"RMSE (all folds): {-scores.mean():.3} ± {(-scores).std():.3}")

RMSE:  [-0.81766997 -1.09084528 -0.71958635 -0.79814608 -0.73148663 -0.77044307]
RMSE (all folds): 0.821 ± 0.125


That is our performance without any additional information added. Therefore, if we do some more feature engineering and add for example whether the day it was tested in is a holiday, a week-end, etc...

In [29]:
results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)

results.to_csv("submission.csv", index=False)